In [1]:
import gensim.downloader as api
import time
from gensim.models import Word2Vec

import sys
sys.path += ['/home/lorenzo/Scrivania/My_projects/DeepWalk/P2Vec/Package/']  

from PWord2Vec import *
from est import *
from p2vec import *

import warnings
warnings.filterwarnings("ignore")

In [2]:
# load the data

dataset = api.load('text8')
text = [a for a in dataset]

In [3]:
# set the parameters
dim = 64
n_epochs = 18
k = 1
window_size = 5
symmetric = True
th = 1e-5
min_count = 5
n_workers = mp.cpu_count()

In [1]:
t0 = time.time()

print("Preprocessing text...")
word2idx, pdrop = PreprocessTextDictionaries(text, th, min_count)

print("Execution time: " + str(time.time() - t0))


t1 = time.time()

print("\nBuilding the matrix P...")
A = build_SG_adjacency_matrix(text, window_size, word2idx, pdrop, n_workers = n_workers)

n = A.shape[0]
d = A@np.ones(n) + 1
D_1 = diags(d**(-1))
P = D_1.dot(A)
Λ = diags(np.log(d))   

print("\nExecution time: " + str(time.time() - t1))

t3 = time.time()
print("\nComputing the embedding...")

w2v = CreateEmbedding([P], Λ = Λ, dim = dim, n_epochs = n_epochs, k = k, symmetric = symmetric)
print("Execution time: " + str(time.time() - t3))

print("\n-------------------------------\n")   
print("Total execution time: " + str(time.time() - t0))

NameError: name 'time' is not defined

In [ ]:
t0 = time.time()
model = Word2Vec(sentences = text, size = dim, negative = 5, window = window_size, min_count = min_count, workers = n_workers)
time.time() - t0

In [ ]:
X = w2v.Φ
how_many = 10
all_words = list(word2idx.keys())

word = 'office'
idx = all_words.index(word)
similarity = X[idx]@X.T
idx = np.argsort(similarity)[::-1]
most_similar = np.array(all_words)[idx][1:how_many+1]
print("Our method:\n")
for m, s in zip(most_similar, similarity[idx][1:how_many+1]):
    print(tuple([m, s]))

print("\n-------------------------\n")
print("Gensim:\n")
for m in model.wv.most_similar(word):
    print(m)

In [ ]:
w2v.plotDistr(all_words.index(word))
n = X.shape[0]
indeces = np.random.choice(np.arange(n), 1000)
kv = [1, 5, 10, 20]
ta = TestApprox(X, indeces, kv)
ta.PlotTest()

Y = model.wv.vectors_norm
ta = TestApprox(Y, indeces, kv)
ta.PlotTest()

In [ ]:
Y = np.array([model.wv.get_vector(w) for w in model.wv.vocab.keys()])
Y = (Y.T - np.reshape(np.mean(Y, axis = 0), (dim, 1))).T        
Y = normalize(Y, norm = 'l2', axis = 1)
        
words_gensim = [w for w in model.wv.vocab.keys()]

In [ ]:
sim = pd.read_csv('/home/lorenzo/Scrivania/My_projects/DeepWalk/P2Vec/data/wordsim353.tsv', header = None, sep = '\t')

os = []
w2vs = []
gts = []

for i in range(len(sim)):
    a = sim.iloc[i][0].lower()
    b = sim.iloc[i][1].lower()
    if (a in all_words) and (b in all_words):
        os.append(X[all_words.index(a)]@X[all_words.index(b)])
        w2vs.append(Y[words_gensim.index(a)]@Y[words_gensim.index(b)])
        gts.append(sim.iloc[i][2])
        
fig, ax = plt.subplots(1, 2, figsize = (10,5))

ax[0].scatter(os, gts)
ax[1].scatter(w2vs, gts)

plt.show();

from scipy.stats import pearsonr, spearmanr


print(pearsonr(os, gts))
print(pearsonr(w2vs, gts))

print("\n----------------")

print(spearmanr(os, gts))
print(spearmanr(w2vs, gts))